# run linear regression
metabolites(Y) ~ the major CANCER lineages(X)

lineage types as binary covariates (X). Cell lines were represented by the rows, with 1 indicating presence of the corresponding feature；Each metabolite level (log10 scale) was used as the response variable （Y）

we also included binary variables indicating major lineages. Here, L1, L2,…, L17 represented the lineages of lung, large intestine, blood, urinary, bone, skin, breast, liver, ovary, oesophagus, endometrium, central nervous system, soft tissue, pancreas, stomach, kidney, and upper aerodigestive tract

In [1]:
name<-load("data/lineage_annot1.Rdat")
name
name<-load("data/metabolomics.Rdat")
name
annot1[1:2,]
dim(annot1)
metabolomics[1:2,]
dim(metabolomics)

library(glmnet)
library(broom)

lr<-lapply(1:(dim(metabolomics)[2]),function(x){
    #remove NA data
    non_NA <- !is.na(metabolomics[,1])
    data.y<-as.numeric(as.character(metabolomics[non_NA,x]))
    data.x<-as.matrix(annot1[non_NA,])
    fit<-lm(data.y~.,data=as.data.frame(data.x))
    #feature=colnames(data_linear)[x]
    pvalue = tidy(summary(fit))$p.value[2]
    r2 = summary(fit)$r.squared
    
    re<-c(r2,pvalue)
    return(re)
  } )


d<-do.call(rbind, lr)
d<-as.data.frame(d)
colnames(d)<-c("R2","pvalue")
dim(d)
d$Metabolites<-colnames(metabolomics)
d<-d[,c(3,1,2)]
d[order(d[,2],decreasing=TRUE),][1:5,]

[1] "annot1"

[1] "metabolomics"

,LUNG,LARGE,HAEMATOPOIETIC,URINARY,BONE,SKIN,BREAST,LIVER,OVARY,OESOPHAGUS,ENDOMETRIUM,CENTRAL,SOFT,PANCREAS,STOMACH,KIDNEY,UPPER
DMS53_LUNG,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SW1116_LARGE_INTESTINE,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


[1] 1072   17

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,C56:8 TAG,C56:7 TAG,C56:6 TAG,C56:5 TAG,C56:4 TAG,C56:3 TAG,C56:2 TAG,C58:8 TAG,C58:7 TAG,C58:6 TAG
DMS53_LUNG,6.112727,6.034198,5.896896,6.000532,5.513619,5.868529,5.977177,5.693074,5.923737,5.641242,...,6.070239,6.133433,6.091089,6.257711,6.372732,6.202511,5.939576,6.309821,6.115974,5.999436
SW1116_LARGE_INTESTINE,5.577413,5.727045,5.111468,6.073250,5.802494,5.824473,5.888821,5.768379,5.760784,5.914742,...,6.248652,6.633575,6.378052,6.341043,6.360945,6.333540,6.137271,7.065858,6.832174,6.363064


[1] 1072  225

Loading required package: Matrix

Loaded glmnet 3.0-2




[1] 225   2

,Metabolites,R2,pvalue
,<chr>,<dbl>,<dbl>
52,phosphocreatine,0.3942951,0.019703870
48,xanthine,0.3633522,0.007720378
190,C20:4 CE,0.3580658,0.001358117
103,creatinine,0.3218722,0.025689697
92,kynurenic acid,0.3208791,0.022114725


# calculate all metabolites correlations vs genetic features

to answer the question that in a given cell line feature matrix (e.g., collections of genetic or epigenetic features) which feature(X,SNP here) was the most associated with a given metabolite vector(Y) under the condition of controlled lineage effects(annot).
The calculated t-statistics, p-values, and estimated coefficients for X (βX) were reported to evaluate the associations.

In [5]:
name<-load("data/lineage_annot1.Rdat")
name
name<-load("data/metabolomics.Rdat")
name
name<-load("data/mut_features.Rdat")
name
annot1[1:2,]
dim(annot1)
metabolomics[1:2,]
dim(metabolomics)
mut_features[1:2,]
dim(mut_features)

[1] "annot1"

[1] "metabolomics"

[1] "mut_features"

,LUNG,LARGE,HAEMATOPOIETIC,URINARY,BONE,SKIN,BREAST,LIVER,OVARY,OESOPHAGUS,ENDOMETRIUM,CENTRAL,SOFT,PANCREAS,STOMACH,KIDNEY,UPPER
DMS53_LUNG,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SW1116_LARGE_INTESTINE,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


[1] 1072   17

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,C56:8 TAG,C56:7 TAG,C56:6 TAG,C56:5 TAG,C56:4 TAG,C56:3 TAG,C56:2 TAG,C58:8 TAG,C58:7 TAG,C58:6 TAG
DMS53_LUNG,6.112727,6.034198,5.896896,6.000532,5.513619,5.868529,5.977177,5.693074,5.923737,5.641242,...,6.070239,6.133433,6.091089,6.257711,6.372732,6.202511,5.939576,6.309821,6.115974,5.999436
SW1116_LARGE_INTESTINE,5.577413,5.727045,5.111468,6.073250,5.802494,5.824473,5.888821,5.768379,5.760784,5.914742,...,6.248652,6.633575,6.378052,6.341043,6.360945,6.333540,6.137271,7.065858,6.832174,6.363064


[1] 1072  225

,MTOR_all_mutations,SPEN_all_mutations,SFPQ_all_mutations,CSF3R_all_mutations,ERBB4_all_mutations,FEV_all_mutations,CACNA1D_all_mutations,MITF_all_mutations,MECOM_all_mutations,PIK3CA_all_mutations,...,EGFR_missense_mutations,HRAS_missense_mutations,SMAD4_missense_mutations,EP300_missense_mutations,FGFR2_missense_mutations,CASP8_missense_mutations,GNAS_missense_mutations,IDH1_missense_mutations,EZH2_missense_mutations,TRRAP_missense_mutations
DMS53_LUNG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SW1116_LARGE_INTESTINE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


[1] 1072  705

In [6]:
library(broom)

get_regression_results_non_order <- function(data.x=data.x,data.y=data.y,input_name=input_name,annot=annot) {
  
  p_result <-rep(NA,length=dim(data.x)[2])
  z_result <-rep(NA,length=dim(data.x)[2])
  est_result <-rep(NA,length=dim(data.x)[2])
  
  j=which(colnames(data.y)==input_name)
  
  for(i in 1:dim(data.x)[2]){
    
    non_NA <- !is.na(data.x[,i]) & !is.na(data.y[,j])
    x <- cbind(data.x[,i],annot)
    lm_result <- lm(data.y[non_NA,j] ~ x[non_NA,])
    p_result[i] <- (tidy(lm_result)$p.value)[2]
    z_result[i] <- (tidy(lm_result)$statistic)[2]
    est_result[i] <- (tidy(lm_result)$estimate)[2]
    
  }
  
  ### 
  q_result <- p.adjust(p_result,method="fdr")
  
  #annotation <- HGNC_table$"Approved.Name"[match(as.character(colnames(data.x)), as.character(HGNC_table$"Approved.Symbol"))]
  result_raw.1 <- cbind(p=p_result, q=q_result, z=z_result, est=est_result)
  rownames(result_raw.1) <- colnames(data.x)
  result <- as.matrix(result_raw.1) 
  return(result)
}



get_array_result <- function(X,Y,sel){
  result <- array(NA,dim=c(dim(X)[2], 4, dim(Y)[2]))
  rownames(result) <- colnames(X)
  colnames(result) <- c("p","q","z","est")
  for(i in 1:dim(Y)[2]){
    #print(i)
    input_name <- colnames(Y)[i]
    result[,,i] <- get_regression_results_non_order(data.x=X[sel,], data.y=Y[sel,], input_name=input_name, annot=annot1[sel,])
  } 
  return(result)
}


In [7]:
#sel_mut <- !is.na(metabolomics[,1]) & !is.na(mutation_all_cosmic[,1])
#input_name="2-aminoadipate"
sel_mut <- !is.na(metabolomics[,1])
out<-get_array_result(X=mut_features, Y=metabolomics, sel=sel_mut)

, , 1

                                          p         q             z
MTOR_all_mutations             0.4158845215 0.8441789  0.8139543183
SPEN_all_mutations             0.2188718980 0.7045876  1.2303842585
SFPQ_all_mutations             0.1980566195 0.6712977  1.2880502461
CSF3R_all_mutations            0.4749699976 0.8488155  0.7147131804
ERBB4_all_mutations            0.0910286055 0.5416970  1.6917927767
FEV_all_mutations              0.3686983376 0.8225707 -0.8993619558
CACNA1D_all_mutations          0.4004259635 0.8431488  0.8412561092
MITF_all_mutations             0.5433361873 0.8777777 -0.6080050179
MECOM_all_mutations            0.3041698206 0.7854935 -1.0281143805
PIK3CA_all_mutations           0.9325290136 0.9749471 -0.0846868562
MAP3K13_all_mutations          0.9292888610 0.9734750 -0.0887642279
TP63_all_mutations             0.2757513187 0.7792776  1.0905703048
WHSC1_all_mutations            0.4559686915 0.8441789  0.7458208194
PDGFRA_all_mutations           0.2692989271 0.7686467 -1.1053542207
TET2_all_mutations             0.0291848050 0.4303687 -2.1844433618
TERT_all_mutations             0.0011811586 0.2081792  3.2537081344
IL6ST_all_mutations            0.1726849859 0.6510316  1.3647014607
EBF1_all_mutations             0.9760428429 0.9906312 -0.0300386337
ROS1_all_mutations             0.1226015650 0.6008919  1.5453851614
BRAF_all_mutations             0.8279837972 0.9601815  0.2173508035
KMT2C_all_mutations            0.8728725647 0.9629379  0.1600559932
IKBKB_all_mutations            0.8988222779 0.9677061 -0.1271848217
PREX2_all_mutations            0.9696958313 0.9893423  0.0380002553
NBN_all_mutations              0.2563294988 0.7444923  1.1358250382
JAK2_all_mutations             0.3639687456 0.8225707  0.9082848522
PTCH1_all_mutations            0.0068598794 0.2646716  2.7097694653
NOTCH1_all_mutations           0.0834421160 0.5299702  1.7329609993
TET1_all_mutations             0.3192333489 0.7903502  0.9965811757
NUP98_all_mutations            0.0512783851 0.4872321  1.9517215624
MEN1_all_mutations             0.7900009796 0.9559758  0.2663878782
MAML2_all_mutations            0.4314656252 0.8441789  0.7870369237
ATM_all_mutations              0.2967286833 0.7806965  1.0440747588
BCL9L_all_mutations            0.1882219083 0.6643152  1.3168477662
FLI1_all_mutations             0.5943087743 0.9017424  0.5327907878
KDM5A_all_mutations            0.4213331674 0.8441789  0.8044751952
KMT2D_all_mutations            0.4049647508 0.8441789  0.8331756942
ERBB3_all_mutations            0.1679487177 0.6434992 -1.3799272681
FLT3_all_mutations             0.4956591307 0.8606889  0.6816104249
BRCA2_all_mutations            0.9037422570 0.9694801 -0.1209691207
NUTM1_all_mutations            0.1818337058 0.6573988  1.3361539742
USP8_all_mutations             0.3399252639 0.8107879  0.9548152366
AXIN1_all_mutations            0.2988850797 0.7827125  1.0394222885
CREBBP_all_mutations           0.1122264654 0.5904452  1.5898067638
PALB2_all_mutations            0.4376843288 0.8441789 -0.7764520519
YWHAE_all_mutations            0.2157651980 0.7009883  1.2387311759
TP53_all_mutations             0.8004731955 0.9581216  0.2528087897
PER1_all_mutations             0.7870152773 0.9559758  0.2702682869
STK11_all_mutations            0.1962334404 0.6683313  1.2933081234
KEAP1_all_mutations            0.0293796639 0.4303687 -2.1818054507
CALR_all_mutations             0.8440213628 0.9629379 -0.1968086005
ASXL1_all_mutations            0.6720017944 0.9217145 -0.4235397688
MYH9_all_mutations             0.0029598263 0.2456892  2.9799448817
P2RY8_all_mutations            0.4294338953 0.8441789  0.7905142855
KDM5C_all_mutations            0.5514829805 0.8783794  0.5957615801
AR_all_mutations               0.6801715788 0.9224253  0.4123618228
MPL_all_mutations              0.5206969376 0.8777777  0.6425213354
NFE2L2_all_mutations           0.5285023812 0.8777777  0.6305363579
ATR_all_mutations              0.0601603855 0.487

In [1]:
library(broom)

name<-load("data/lineage_annot1.Rdat")
name
name<-load("data/metabolomics.Rdat")
name
name<-load("data/mut_features.Rdat")
name
name<-load("data/meth_features.Rdat")
name
name<-load("data/CN_features.Rdat")
name
sel_mut <- !is.na(metabolomics[,1])
data.x=meth_features[sel_mut,]
data.y=metabolomics[sel_mut,]
annot=annot1[sel_mut,]
x=cbind(data.x[,"AADAT_methylation"],annot)
lm_result <- lm(data.y[,"3-phosphoglycerate"] ~ x)
#summary(lm_result)

#(tidy(lm_result)$p.value)[2]
(tidy(lm_result)$statistic)[2]
#(tidy(lm_result)$estimate)[2]

[1] "annot1"

[1] "metabolomics"

[1] "mut_features"

[1] "meth_features"

[1] "CN_features"

[1] -1.186952

In [6]:
d<-CN_features
d<-na.omit(d)
dim(d)

write.csv(d,file="CN_features.txt",quote=F,row.names=T)


[1] 1030   61